In [1]:
import datetime
import json
import random
import torch
import os

from transformers import pipeline

# Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/MyDrive/Spring 2024/nyt_puzzles/nyt_connect/puzzles.json') as f:
    puzzle_data = json.load(f)

# Helper function

In [10]:
def date_to_puzzle(date):
    first_puzzle_id = 204
    first_date = datetime.datetime.strptime('01/01/2024','%m/%d/%Y')

    number_of_days = (date - first_date)
    return first_puzzle_id + number_of_days.days

def load_puzzle(puzzle_data, puzzle_id):
    for puzzle in puzzle_data:
        # print(puzzle['puzzle_id'])
        if puzzle['puzzle_id'] == str(puzzle_id):
            return puzzle['content']
    return None

def load_words(puzzle_content):
    answers, words = {}, []
    for group in puzzle_content:
        answers[group['description']] = group['words']
        words += group['words']
    return answers, words

def llm_suggestion(pipe, words):

    word_lst = ', '.join(words)

    messages = [
        {
            "role": "system",
            "content": '''
                You are a human trying to solve a puzzle using your knowledge on different word meanings, slangs, idioms, and popular culture references
            ''',
        },
        {
            "role": "user",
            "content": f'''
                Given the list of words: {word_lst}

                Solve the puzzle by following these exact rules:
                - Each word in the list of words above belongs to exactly 1 unspecified category. There are 4 categories in total with each category containing 4 different words
                - Determine the 4 categories and the 4 respective words belonging to them
                - Return the solution of this puzzle in the exact format below:

                    Category 1: [4 words from the list]
                    Category 2: [4 words from the list]
                    Category 3: [4 words from the list]
                    Category 4: [4 words from the list]


            '''},
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    res = outputs[0]["generated_text"]
    return res.split('<|assistant|>', 1)[1].lower()

# Main Program

In [5]:
date_str = input('Choose date (MM/DD/2024): ')
date = datetime.datetime.strptime(date_str, '%m/%d/%Y')

puzzle_id = date_to_puzzle(date)
print('Puzzle Number: ', puzzle_id)

Choose date (MM/DD/2024): 01/02/2024
Puzzle Number:  205


In [6]:
content = load_puzzle(puzzle_data, puzzle_id)
answers, words = load_words(content)

random.shuffle(words)

print(f'\n-----Puzzle {puzzle_id} - {date_str}-----')
for i in range(4):
    print('| ' + ' | '.join(words[i*2: i*2 + 4]) + ' |')



-----Puzzle 205 - 01/02/2024-----
| JAM | DOUBLE | WONDER | CLOG |
| WONDER | CLOG | HIT | FRY |
| HIT | FRY | WORLD | RUN |
| WORLD | RUN | WALK | TOMATO |


In [11]:
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v0.6", torch_dtype=torch.bfloat16)

In [13]:
print('\n----- GenAI Suggestion -----')
print(llm_suggestion(pipe, words))


----- GenAI Suggestion -----

to solve the puzzle using the above rules, follow these steps:

1. first, identify the categories: determine the 4 categories in the puzzle, which are jam, double, wonder, clog, hit, fry, world, run, walk, tomato, bacon, talk, lettuce, stop, block, bread.

2. identify the words belonging to each category: determine the 4 words from each category that belong to the puzzle. for example, jam belongs to category 1, double belongs to category 2, wonder belongs to category 3, clog belongs to category 4, hit belongs to category 3, fry belongs to category 2, world belongs to category 3, run belongs to category 1, walk belongs to category 2, tomato belongs to category 3, bacon belongs to category 1, talk belongs to category 1, lettuce belongs to category 4, stop belongs to category 2, block belongs to category


In [8]:
print(f'\n----- Answer -----')
for group in answers.keys():
    print(f'{group}: '+ ', '.join(answers[group]))


----- Answer -----
B.L.T. INGREDIENTS: BACON, BREAD, LETTUCE, TOMATO
OBSTRUCT: BLOCK, CLOG, JAM, STOP
BASEBALL STATS: DOUBLE, HIT, RUN, WALK
SMALL ___: FRY, TALK, WONDER, WORLD
